# `holodeck` - Semi-Analytic Models

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm

import kalepy as kale

import holodeck as holo
from holodeck import sams
from holodeck import utils, plot
from holodeck.constants import MSOL, YR

## Quick Start

Construct a Semi-Analytic Model (SAM) using all of the default components

In [ ]:
# Specify the shape of the SAM grid to be a small number (e.g. `30`), so that this example runs quickly
# (although with low accuracy).
sam = sams.Semi_Analytic_Model(shape=30)

Choose the edges of the frequency bins at which to calculate the GWB

In [ ]:
OBS_DUR = 10.0 * YR    # duration of PTA observations in [sec], which determines the Fourier frequency basis
NUM_FREQS = 20         # number of frequency bins
fobs, fobs_edges = utils.pta_freqs(dur=OBS_DUR, num=NUM_FREQS)
print(f"Number of frequency bins: {fobs.size}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

Calculate GWB from this SAM.  
We need to specify the edges of the frequency bins that are being observed (`fobs_edges`).  
We also ask for many different 'realizations' of the universe to get a distribution of expected amplitudes.  
And finally we will obtain a handful of the 'loudest' binaries in each frequency bin, ('single sources'),  in addition to the sum of the characteristic strains of all remaining binaries (the background).


In [ ]:
NUM_REALS = 30    # Number of 'realizations' to generate
NUM_LOUDEST = 2   # Number of 'loudest' binaries to generate in each frequency bin
hc_ss, hc_bg = sam.gwb(fobs_edges, realize=NUM_REALS, loudest=NUM_LOUDEST)

Plot GWB over multiple realizations

In [ ]:
plt.loglog(fobs*1e9, hc_bg, lw=0.5, alpha=0.75);
plt.gca().set(ylabel='Characteristic Strain ($h_c$)', xlabel='GW Frequency [nHz]')
plt.show()

Slightly fancier plot:

In [ ]:
fig, ax = plot.figax(xlabel='GW Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

# `fobs` are bin centers in CGS units, convert to [1/yr]
xx = fobs * YR

# Get the median over all the realizations
med = np.median(hc_bg, axis=-1)

# plot a reference, pure power-law  strain spectrum:   h_c(f) = A * (f * yr) ^ -2/3
yy = med[0] * np.power(xx/xx[0], -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)

# Plot the median GWB spectrum
ax.plot(xx, med, 'k-', alpha=0.5)

# plot contours at 50% and 98% confidence intervals
for pp in [50, 98]:
    percs = pp / 2
    percs = [50 - percs, 50 + percs]
    ax.fill_between(xx, *np.percentile(hc_bg, percs, axis=-1), alpha=0.25, color='#7100d4')

plt.show()

## Build SAM with explicit components

In [ ]:
gsmf = holo.sams.GSMF_Schechter()               # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sams.GPF_Power_Law()                 # Galaxy Pair Fraction         (GPF)
gmt = holo.sams.GMT_Power_Law()                 # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_MM2013()       # M-MBulge Relation            (MMB)

sam = holo.sams.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=30)

Calculate the distribution of GWB Amplitudes at 1/yr

In [ ]:
# Choose our target GW frequency
fobs = 1.0/YR
# Choose an appropriate bin width
fobs_width = fobs/16.0
fobs_edges = np.array([fobs - 0.5*fobs_width, fobs + 0.5*fobs_width])
hc_ss, hc_bg = sam.gwb(fobs_edges, realize=300, loudest=1)
# Calculate the idealized GWB amplitude from this population
gwb_ref = sam.gwb_ideal(fobs)

amp_bg = hc_bg.flatten()
# combine the single-sources and background sources into total amplitude
amp_tot = np.sum(hc_ss**2, axis=-1) + hc_bg**2
amp_tot = np.sqrt(amp_tot).flatten()

Plot the distribution

In [ ]:
fig, ax = plt.subplots(figsize=[8, 4])
ax.set(xlabel=r'$\log_{10}(A_\mathrm{yr})$', ylabel='Probability Density')
ax.grid(alpha=0.2)

# use `kalepy` do draw the 1D distribution
h1 = kale.dist1d(np.log10(amp_tot), density=True, confidence=True)
h2 = kale.dist1d(np.log10(amp_bg), density=True, confidence=False)
h3 = ax.axvline(np.log10(gwb_ref), ls='--', color='k')
ax.legend([h1, h2, h3], ['Total', 'BG only', 'idealized'])

plt.show()

## Plot GWB Amplitude Distribution vs. M-MBulge parameters

Calculate GWB amplitudes at $f = 1/yr$ over a grid of M-Mbulge parameters, specifically the amplitude and power-law.

In [ ]:
# Choose parameters to explore
NREALS = 10     # number of realizations at each parameter combination
alpha_list = [1.0, 1.5]     # M-Mbulge power-law index
norm_list = [3e8, 3e9]
# norm_list = np.logspace(8, 9.5, 4)     # M-Mbulge normalization, units of [Msol]

dist_mmb = np.zeros((len(alpha_list), len(norm_list), NREALS))

# Iterate over a grid of both paremeters
for aa, alpha in enumerate(tqdm.tqdm(alpha_list)):
    for nn, norm in enumerate(tqdm.tqdm(norm_list, leave=False)):
        # Create the M-Mbulge relationship for these parameters
        mmbulge = holo.relations.MMBulge_Standard(mamp=norm*MSOL, mplaw=alpha)
        # Build a new sam
        sam = holo.sams.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=20)
        # Calculate the distribution of GWB amplitudes
        cw, dist_mmb[aa, nn, :] = sam.gwb(fobs_edges, realize=NREALS, loudest=0)
        if nn == 0:
            print(f"{aa=} {alpha=} {np.median(dist_mmb[aa, nn, :]):.4e}")
        # break

Plot the interquartile ranges for each power-law, as a function of normalization

In [ ]:
fig, ax = plt.subplots(figsize=[6, 4])
ax.set(xscale='log', xlabel='M-MBulge Mass Normalization', yscale='log', ylabel=r'GWB Amplitude $A_\mathrm{yr}$')
ax.grid(alpha=0.2)

for aa, dd in zip(alpha_list, dist_mmb):
    med = np.median(dd, axis=-1)
    cc, = ax.plot(norm_list, med, label=aa)
    cc = cc.get_color()
    ax.fill_between(norm_list, *np.percentile(dd, [25, 75], axis=-1), color=cc, alpha=0.15)

plt.legend(title='M-MBulge Slope')
plt.show()